Mounted at /content/gdrive


In [2]:
#Connect to data folder of NerLTR-DTA
root_path = '/data/'
import os
os.chdir(root_path)
!pwd

/content/gdrive/My Drive/KI JKU/Seminar_PracticalW_MasterThesis/MasterThesis/NerLTR/data


In [3]:
import numpy as np
import pandas as pd
import linecache
import math
from collections import Counter
import pickle
from ast import literal_eval

#get Folds for affinity and drug and protein Ids

In [4]:
path_KIBA_affinity = 'KIBA/kiba_binding_affinity_v2.txt' #Affinity Matrix AffiDT from Kiba Dataset
path_KIBA_fold_train = 'KIBA/fold/train_fold_setting1.txt'
path_KIBA_fold_test = 'KIBA/fold/test_fold_setting1.txt'

In [5]:
with open(path_KIBA_fold_test) as f:
    test_KIBA = literal_eval(f.read())
print(f'Test {len(test_KIBA)}')

Test 19709


In [6]:
with open(path_KIBA_fold_train) as f:
    train_KIBA = literal_eval(f.read())
print(f'Fold1 {len(train_KIBA[0])}')
print(f'Fold2 {len(train_KIBA[1])}')
print(f'Fold3 {len(train_KIBA[2])}')
print(f'Fold4 {len(train_KIBA[3])}')
print(f'Fold5 {len(train_KIBA[4])}')
#arr[0] # check ids if fold with data fold file

Fold1 19709
Fold2 19709
Fold3 19709
Fold4 19709
Fold5 19709


In [7]:
#load Affinity Matrix
aff_KIBA = np.loadtxt(path_KIBA_affinity)

In [8]:
# counter to check how many elements in dataset are not nan e.g. KIBA 118254
counter = 0
#dataframe for Drug ID, Target ID plus Affinity and general ID 
kiba_df = pd.DataFrame(columns = ['drug_ID', 'target_ID', 'affinity'])
for i,row in enumerate(aff_KIBA):
  for j,el in enumerate(row):
    if el > -1:
      counter +=1
      data = {
        'drug_ID':i,
        'target_ID':j,
        'affinity': el
        }

      kiba_df=kiba_df.append(data, ignore_index=True)
kiba_df.drug_ID = kiba_df.drug_ID.astype(int)
kiba_df.target_ID = kiba_df.target_ID.astype(int)
print(f'Counter: {counter}')

Counter: 118254


In [12]:
affi_train = kiba_df.iloc[train_KIBA[0]+train_KIBA[1]+train_KIBA[2]+train_KIBA[3]+train_KIBA[4]].copy()
affi_fold1 = kiba_df.iloc[train_KIBA[0]].copy()
affi_fold2 = kiba_df.iloc[train_KIBA[1]].copy()
affi_fold3 = kiba_df.iloc[train_KIBA[2]].copy()
affi_fold4 = kiba_df.iloc[train_KIBA[3]].copy()
affi_fold5 = kiba_df.iloc[train_KIBA[4]].copy()
affi_test = kiba_df.iloc[test_KIBA].copy()

In [13]:
affi_train

,drug_ID,target_ID,affinity
113703,1776,20,9.798970
51222,492,160,11.400000
98078,1207,204,11.200000
29622,305,98,11.200000
80976,917,46,11.999998
...,...,...,...
88379,1017,96,13.800057
98714,1218,39,11.600000
11884,158,3,11.500000
90741,1080,36,10.600000


#Get Similaritys

In [9]:
path_KIBA_drug_sim = 'KIBA/kiba_drug_sim.txt' #Drug Similarity Matrix SimDD from Kiba Dataset
path_KIBA_target_sim = 'KIBA/kiba_target_sim.txt' #Target Similarity Matrix SimTT from Kiba Dataset

In [10]:
def get_features(drug_sim_path, target_sim_path, fold):
  drug_sim = np.loadtxt(drug_sim_path)
  target_sim = np.loadtxt(target_sim_path)
  feature_list=[]
  for index, row in fold.iterrows():
   drug_feature = drug_sim[int(row['drug_ID'])]
   target_feature = target_sim[int(row['target_ID'])]

   feature_list.append(np.concatenate((drug_feature, target_feature)))
  return feature_list


In [11]:
features_fold1 = get_features(path_KIBA_drug_sim, path_KIBA_target_sim, affi_fold1)
features_fold2 = get_features(path_KIBA_drug_sim, path_KIBA_target_sim, affi_fold2)
features_fold3 = get_features(path_KIBA_drug_sim, path_KIBA_target_sim, affi_fold3)
features_fold4 = get_features(path_KIBA_drug_sim, path_KIBA_target_sim, affi_fold4)
features_fold5 = get_features(path_KIBA_drug_sim, path_KIBA_target_sim, affi_fold5)

NameError: ignored

In [ ]:
features_test = get_features(path_KIBA_drug_sim, path_KIBA_target_sim, affi_test)

#Write Files in the RankLibFormat

In [ ]:
def write_file(outpath:str, drug_sim_path, target_sim_path, fold):
  with open(outpath, 'w') as f:
    drug_sim = np.loadtxt(drug_sim_path)
    target_sim = np.loadtxt(target_sim_path)
    for index, row in fold.iterrows():
      #write affinity
      f.write(str(row['affinity']))
      #write qid
      f.write(' qid:')
      f.write(str(int(row['drug_ID'])))
      f.write(' ')
      #write features
      feature_number = 1  # for Ranklib thte feature number has to be infront of each feature
      #features
      #Drugs
      drug_feature = drug_sim[int(row['drug_ID'])]
      for el in drug_feature:
        f.write(str(feature_number)+':')
        f.write(str(el))
        f.write(' ')
        feature_number +=1
      #Targets
      target_feature = target_sim[int(row['target_ID'])]
      for el in target_feature:
        f.write(str(feature_number)+':')
        f.write(str(el))
        f.write(' ')
        feature_number +=1
      f.write('\n')
      

In [ ]:
write_file('features/combined_similarities/train_fold1_KIBA.txt', path_KIBA_drug_sim, path_KIBA_target_sim, affi_fold1)
write_file('features/combined_similarities/train_fold2_KIBA.txt', path_KIBA_drug_sim, path_KIBA_target_sim, affi_fold2)
write_file('features/combined_similarities/train_fold3_KIBA.txt', path_KIBA_drug_sim, path_KIBA_target_sim, affi_fold3)
write_file('features/combined_similarities/train_fold4_KIBA.txt', path_KIBA_drug_sim, path_KIBA_target_sim, affi_fold4)
write_file('features/combined_similarities/train_fold5_KIBA.txt', path_KIBA_drug_sim, path_KIBA_target_sim, affi_fold5)

In [ ]:
write_file('train_KIBA_sim.txt', path_KIBA_drug_sim, path_KIBA_target_sim, affi_train)
write_file('test_KIBA_sim.txt', path_KIBA_drug_sim, path_KIBA_target_sim, affi_test)

#Write Files separate for Drugs and Targets without RankLib Format

In [ ]:
def write_file_drugs(outpath:str, drug_sim_path, fold):
    drug_sim = np.loadtxt(drug_sim_path)
    features = []
    for index, row in fold.iterrows():
      #Drugs
      drug_feature = drug_sim[int(row['drug_ID'])]
      features.append(drug_feature)
    features_np = np.asarray(features)
    np.save(outpath, features_np)

def write_file_targets(outpath:str, target_sim_path, fold):
    target_sim = np.loadtxt(target_sim_path)
    features = []
    for index, row in fold.iterrows():
      #Targets
      target_feature = target_sim[int(row['target_ID'])]
      features.append(target_feature)
    features_np = np.asarray(features)
    np.save(outpath, features_np)

In [ ]:
write_file_drugs('features/learned/train_fold1_drugs.npy', path_KIBA_drug_sim, affi_fold1)
write_file_drugs('features/learned/train_fold2_drugs.npy', path_KIBA_drug_sim, affi_fold2)
write_file_drugs('features/learned/train_fold3_drugs.npy', path_KIBA_drug_sim, affi_fold3)
write_file_drugs('features/learned/train_fold4_drugs.npy', path_KIBA_drug_sim, affi_fold4)
write_file_drugs('features/learned/train_fold5_drugs.npy', path_KIBA_drug_sim, affi_fold5)

In [ ]:
write_file_targets('features/learned/train_fold1_targets.npy', path_KIBA_target_sim, affi_fold1)
write_file_targets('features/learned/train_fold2_targets.npy', path_KIBA_target_sim, affi_fold2)
write_file_targets('features/learned/train_fold3_targets.npy', path_KIBA_target_sim, affi_fold3)
write_file_targets('features/learned/train_fold4_targets.npy', path_KIBA_target_sim, affi_fold4)
write_file_targets('features/learned/train_fold5_targets.npy', path_KIBA_target_sim, affi_fold5)

In [27]:
def write_file_full_rf(outpath:str, drug_sim_path, target_sim_path, fold):

  drug_sim = np.loadtxt(drug_sim_path)
  target_sim = np.loadtxt(target_sim_path)
  features_full = []
  for index, row in fold.iterrows():
    #features
    features = []
    #Drugs
    drug_feature = drug_sim[int(row['drug_ID'])]
    for el in drug_feature:
      features.append(el)
    #Targets
    target_feature = target_sim[int(row['target_ID'])]
    for el in target_feature:
      features.append(el)
    features_full.append(features)
  features_np = np.asarray(features_full)
  print(len(features_np))
  #np.save(outpath, features_np)
  np.savetxt(outpath, features_np, delimiter=',') 

In [28]:
write_file_full_rf('train_KIBA_sim_RF', path_KIBA_drug_sim,  path_KIBA_target_sim, affi_train)
write_file_full_rf('test_KIBA_sim_RF', path_KIBA_drug_sim,  path_KIBA_target_sim, affi_test)

98545
19709
